In [1]:
import artm

batch_vectorizer = artm.BatchVectorizer(data_path='.', data_format='bow_uci',
                                        collection_name='kos', target_folder='kos_batches')

In [ ]:
from string import punctuation, ascii_letters
def cleanAndSplit(doc: str) -> [str]:
    nopunctuation = [character for character in doc1.lower() if character in ascii_letters or character == ' ']
    return "".join(nopunctuation).split()

In [181]:
doc1 = """
If you can keep your head when all about you   
    Are losing theirs and blaming it on you,   
If you can trust yourself when all men doubt you,
    But make allowance for their doubting too;   
If you can wait and not be tired by waiting,
    Or being lied about, don’t deal in lies,
Or being hated, don’t give way to hating,
    And yet don’t look too good, nor talk too wise:
"""
doc2 = """
If you can dream—and not make dreams your master;   
    If you can think—and not make thoughts your aim;   
If you can meet with Triumph and Disaster
    And treat those two impostors just the same;   
If you can bear to hear the truth you’ve spoken
    Twisted by knaves to make a trap for fools,
Or watch the things you gave your life to, broken,
    And stoop and build ’em up with worn-out tools:
"""
doc3 = """
If you can make one heap of all your winnings
    And risk it on one turn of pitch-and-toss,
And lose, and start again at your beginnings
    And never breathe a word about your loss;
If you can force your heart and nerve and sinew
    To serve your turn long after they are gone,   
And so hold on when there is nothing in you
    Except the Will which says to them: ‘Hold on!’
"""

In [196]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim

ModuleNotFoundError: No module named 'pyLDAvis'

In [197]:
stopwords

<WordListCorpusReader in '/Users/igorpostoev/nltk_data/corpora/stopwords'>

In [190]:
df = pd.read_csv('voted-kaggle-dataset.csv')

In [200]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [192]:
print(df['Description'].head(2))

0    The datasets contains transactions made by cre...
1    The ultimate Soccer database for data analysis...
Name: Description, dtype: object


In [208]:
texts = []

# loop through document list
for i in df['Description'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # remove stop words from tokens
    #stopped_tokens_new = [raw for raw in stopped_tokens if not raw in en_stop]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)
print(len(texts[0]))

164


In [212]:
print(len(texts))

2885


In [246]:
docs = texts
vocab = list(set([word for doc in docs for word in doc]))
#later - filter by tfidf

In [248]:
D = len(docs)
V = len(vocab)
T = 5
imax = 1

In [221]:
import numpy as np

In [249]:
PHI = np.random.rand(V, T)
THETA = np.random.rand(T, D)
DW = [[doc.count(word) for word in vocab] for doc in docs]
DW = np.array(DW)
lens = np.array([len(d) for d in docs])

In [250]:
for _ in range(imax):
    nwt = np.zeros((V, T))
    ntd = np.zeros((T, D))
    nt = np.zeros((T))
    product = PHI.dot(THETA)
    for d in range(D):
        for w in range(V):
            if product[w][d] == 0:
                ptdw = 0
            else:
                ptdw = PHI[w] * THETA.T[d] / product[w][d] # 1 x T
            ntdw = DW[d][w] * ptdw
            nwt[w] += ntdw
            ntd.T[d] += ntdw
            nt += ntdw
    PHI = nwt / nt # W X T
    THETA = (ntd / lens) # T x D

In [256]:
PHI.T[0].shape

(30922,)

In [259]:
PHI_max_indeces = np.argsort(PHI.T[0])[::-1][:10]

In [260]:
PHI_max_indeces

array([15936, 12715,  8313, 15919, 23159, 20232, 30729, 24322, 10675,
        4460])

In [283]:
def show_semantic_core(topic_index: int, core_len: int):
    print(f"topic {topic_index}: \n")
    PHI_max_indeces = np.argsort(PHI.T[topic_index])[::-1][:core_len]
    id_to_word = dict([(index, word) for index, word in enumerate(vocab)])
    [print(id_to_word[index]) for index in PHI_max_indeces]
    print("\n")

In [282]:
[show_semantic_core(index, 10) for index in range(T)]

topic 0: 

dataset
data
http
name
com
type
number
context
one
bea


topic 1: 

data
time
number
csv
name
inspiration
context
http
model
information


topic 2: 

context
data
csv
file
acknowledgement
code
time
like
content
university


topic 3: 

data
dataset
file
content
name
inspiration
information
year
com
total


topic 4: 

data
contains
information
dataset
content
acknowledgement
context
state
number
type




[None, None, None, None, None]

In [97]:
a = np.array([[2, 4, 6], [2, 4, 6], [2, 4, 6]])

In [98]:
b = np.array([[2, 4, 6], [2, 4, 6], [2, 4, 6]])

In [99]:
a / b

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [2]:
lda = artm.LDA(num_topics=15, alpha=0.01, beta=0.001, cache_theta=True,
               num_document_passes=5, dictionary=batch_vectorizer.dictionary)

In [3]:
lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

In [6]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
    print("Topic #{0}: {1}".format(i, token_list))

Topic #0: ['bush', 'party', 'tax', 'president', 'campaign', 'political', 'state', 'court', 'republican', 'states']
Topic #1: ['iraq', 'war', 'military', 'troops', 'iraqi', 'killed', 'soldiers', 'people', 'forces', 'general']
Topic #2: ['november', 'poll', 'governor', 'house', 'electoral', 'account', 'senate', 'republicans', 'polls', 'contact']
Topic #3: ['senate', 'republican', 'campaign', 'republicans', 'race', 'carson', 'gop', 'democratic', 'debate', 'oklahoma']
Topic #4: ['election', 'bush', 'specter', 'general', 'toomey', 'time', 'vote', 'campaign', 'people', 'john']
Topic #5: ['kerry', 'dean', 'edwards', 'clark', 'primary', 'democratic', 'lieberman', 'gephardt', 'john', 'iowa']
Topic #6: ['race', 'state', 'democrats', 'democratic', 'party', 'candidates', 'ballot', 'nader', 'candidate', 'district']
Topic #7: ['administration', 'bush', 'president', 'house', 'years', 'commission', 'republicans', 'jobs', 'white', 'bill']
Topic #8: ['dean', 'campaign', 'democratic', 'media', 'iowa', 's

In [15]:
lda.get_theta().iloc[:10]

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
topic_0,0.071573,0.000340,0.064975,0.329290,0.042682,0.000143,0.110372,0.000344,0.000585,0.000337,...,0.001335,0.000568,0.000612,0.000105,0.133693,0.004082,0.304049,0.000268,0.034589,0.000489
topic_1,0.005121,0.000258,0.000278,0.000223,0.000398,0.000084,0.001283,0.000294,0.000199,0.000178,...,0.000197,0.000390,0.096175,0.003757,0.143722,0.000193,0.025600,0.179343,0.000151,0.000695
topic_2,0.000078,0.037427,0.000186,0.000178,0.000492,0.769481,0.000168,0.000211,0.000271,0.000214,...,0.046747,0.008441,0.000115,0.520337,0.000168,0.693860,0.000151,0.000146,0.000069,0.000234
topic_3,0.001420,0.000187,0.005598,0.359139,0.007767,0.000205,0.405234,0.146971,0.109285,0.031778,...,0.004812,0.000369,0.000410,0.000127,0.010107,0.000186,0.002427,0.000654,0.000404,0.000501
topic_4,0.462315,0.003936,0.009519,0.011956,0.000661,0.000124,0.000257,0.000325,0.000228,0.000577,...,0.000597,0.000398,0.001485,0.000106,0.000721,0.000210,0.000549,0.001168,0.000120,0.001811
topic_5,0.000110,0.000151,0.001382,0.039472,0.066723,0.000432,0.000144,0.000719,0.000530,0.005010,...,0.000700,0.000433,0.022157,0.000887,0.000134,0.000302,0.000151,0.000913,0.000219,0.000681
topic_6,0.000447,0.000182,0.004930,0.000325,0.000607,0.000140,0.413233,0.425574,0.244290,0.002977,...,0.006587,0.000376,0.000465,0.110521,0.085259,0.002226,0.001881,0.012912,0.027951,0.001180
topic_7,0.000324,0.000474,0.000343,0.000231,0.000377,0.000097,0.002202,0.000283,0.000266,0.000671,...,0.023712,0.000561,0.000194,0.000104,0.042318,0.000108,0.421977,0.153726,0.000137,0.000306
topic_8,0.000149,0.000210,0.000440,0.005485,0.000598,0.000155,0.000120,0.000374,0.000324,0.000637,...,0.000276,0.000411,0.000706,0.000415,0.000107,0.014165,0.000196,0.000346,0.000183,0.000508
topic_9,0.038160,0.955370,0.000451,0.211434,0.508272,0.000176,0.019946,0.423294,0.642150,0.952100,...,0.000303,0.958424,0.001431,0.000085,0.223766,0.000486,0.234543,0.297471,0.073092,0.000716


In [3]:
!export ARTM_SHARED_LIBRARY=/Users/igorpostoev/Projects/Postgraduate/bigartm/build/lib

In [4]:
import pandas

In [1]:
import platform
platform.platform()

'macOS-12.6.2-arm64-arm-64bit'

In [2]:
!export ARTM_SHARED_LIBRARY=/usr/local/lib/libartm.dylib

In [10]:
!pip install artm

ERROR: Could not find a version that satisfies the requirement artm (from versions: none)
ERROR: No matching distribution found for artm

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
